In [26]:
# ! pip install checklist
# ! pip install nltk
# ! pip install pandas
# ! pip install simpletransformers


  Using cached cachetools-5.0.0-py3-none-any.whl (9.1 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=a5a6ca3f32ff322c2852e510e70506d25476ff8517d8f780a03e070b5bc602cb
  Stored in directory: c:\users\valen\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a64cfe95a48e530d3d965898126427b9dd031f7d6b29116e2872b34fdb046c7c
  Stored in dir

In [1]:
import nltk
# nltk.download('omw-1.4')

In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel

np.random.seed(42)

editor = Editor()

In [11]:
model = ClassificationModel("bert", "outputs", use_cuda = False)


In [43]:
subset = pd.read_csv("data\olid-subset-diagnostic-tests.csv", index_col = 0)
subset

,text,labels
id,,
89200,@USER @USER Who the hell does he think he is?,1
71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1
55633,"#OrrinHatch I can’t believe this sexist , clue...",1
16856,@USER @USER I'll use that one the next time im...,1
26072,0-1 lost my acca on the first fucking fight cba,1
...,...,...
45518,@USER He is obviously getting suspended. He is...,0
51610,#Canada - EXCLUSIVE: #Trudeau #Liberals leave ...,0
26758,@USER @USER ...than why did you show us how ho...,0


In [14]:
data = subset["text"].tolist()
ret = Perturb.perturb(data, Perturb.add_typos)
ret.data[0]

['@USER @USER Who the hell does he think he is?',
 '@USER @USER Who the hell does he thinkh e is?']

In [18]:
ret.data[0][1]

'@USER @USER Who the hell does he thinkh e is?'

In [20]:
pred = model.predict([ret.data[0][1]])
pred[1]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([[-2.95100451,  2.09281111]])

# Answer Questions

In [57]:
data = subset["text"].tolist()
ret = Perturb.perturb(data, Perturb.add_typos, keep_original=False, typos=3)
ret.data[0]

['@USER @USER Who the ehll doesh e htink he is?']

In [58]:
subset_perturbed = subset.copy()
subset_perturbed["text"] = ret.data
subset_perturbed

,text,labels
id,,
89200,[@USER @USER Who the ehll doesh e htink he is?],1
71294,[#BREAKING. #Greece: Molotovc ocktails fly aft...,1
55633,"[#OrirnHatch I can’t believe this sxeist , clu...",1
16856,[@USER @USER I'llu se that one the next time i...,1
26072,[0-1 lost my acca on the firtsf ucking fightc ba],1
...,...,...
45518,[@USER He is obvioulsy getting suspende.d He i...,0
51610,[#Canada - EXLCUSIVE: #Trudeau #Liberals leav ...,0
26758,[@USER @SUER ...thanw hy didy ou show us how h...,0


In [59]:
subset

,text,labels
id,,
89200,@USER @USER Who the hell does he think he is?,1
71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1
55633,"#OrrinHatch I can’t believe this sexist , clue...",1
16856,@USER @USER I'll use that one the next time im...,1
26072,0-1 lost my acca on the first fucking fight cba,1
...,...,...
45518,@USER He is obviously getting suspended. He is...,0
51610,#Canada - EXCLUSIVE: #Trudeau #Liberals leave ...,0
26758,@USER @USER ...than why did you show us how ho...,0


In [60]:
results_perturbed = model.eval_model(subset_perturbed)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [47]:
results = model.eval_model(subset)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [61]:
results_perturbed

({'mcc': 0.39834823795432894,
  'tp': 27,
  'tn': 42,
  'fp': 8,
  'fn': 23,
  'auroc': 0.7988,
  'auprc': 0.8227986524547432,
  'eval_loss': 1.633817094575184},
 array([[ 3.35549498, -3.04552531],
        [ 3.2291944 , -2.74314833],
        [-3.00356579,  2.12855554],
        [ 2.34836817, -1.71408772],
        [ 3.5135355 , -3.21535587],
        [-3.13082647,  2.21153116],
        [ 3.66914988, -3.34923983],
        [-1.84804022,  1.27670848],
        [ 3.60711479, -3.32378817],
        [-2.98464751,  2.11612511],
        [ 3.31171918, -2.82654858],
        [ 3.58604813, -3.24517274],
        [ 0.59043586, -0.39974344],
        [ 3.50623608, -3.28191972],
        [-3.06002378,  2.16109419],
        [-3.13924837,  2.21356535],
        [ 3.36733532, -2.67475653],
        [-2.79181552,  1.99612904],
        [-2.32221723,  1.65706956],
        [-3.08523273,  2.18337965],
        [-2.89093661,  2.04987121],
        [-3.09303164,  2.18989801],
        [-2.7788415 ,  1.98262775],
        [ 

In [49]:
results

({'mcc': 0.530722777603022,
  'tp': 33,
  'tn': 43,
  'fp': 7,
  'fn': 17,
  'auroc': 0.844,
  'auprc': 0.864618820961216,
  'eval_loss': 1.2756656694154327},
 array([[-3.06474948,  2.16581249],
        [ 3.6282208 , -3.22378826],
        [-3.15027356,  2.22297668],
        [ 2.93057585, -2.16962624],
        [-3.10141563,  2.19616413],
        [-3.15645051,  2.2372098 ],
        [ 3.68977094, -3.34390259],
        [-1.48650384,  1.04254591],
        [ 3.07217693, -2.46045327],
        [-3.09795117,  2.19029284],
        [ 3.14411044, -2.39236736],
        [-3.07784128,  2.18293166],
        [-3.08340716,  2.19401503],
        [-2.95433807,  2.06923723],
        [-3.12407637,  2.20752764],
        [-3.16261768,  2.24090171],
        [-3.08751488,  2.20339942],
        [-3.10409737,  2.17998123],
        [-3.08633733,  2.18803453],
        [-3.16389847,  2.25638437],
        [-3.03999472,  2.15788507],
        [-3.13396788,  2.22007346],
        [-3.115417  ,  2.19255066],
        [ 3.6

In [63]:
fn = results[0]['fn']
fp = results[0]['fp']
tn = results[0]['tn']
tp = results[0]['tp']

print('Class 0:\n')

precision = tn/(tn+fn)
print(f'Precision: {precision}')

recall = tn/(tn+fp)
print(f'Recall: {recall}')

f10 = 2*((precision*recall)/(precision+recall))
print(f'F1: {f10}')

print()
print()

print('Class 1:\n')

precision = tp/(tp+fp)
print(f'Precision: {precision}')

recall = tp/(tp+fn)
print(f'Recall: {recall}')

f11 = 2*((precision*recall)/(precision+recall))
print(f'F1: {f11}')

print()
print()

print(f'Weighted avg F1: {(620*f10+240*f11)/860}')
print(f'Macro avg F1: {0.5*f10+0.5*f11}')

Class 0:

Precision: 0.7166666666666667
Recall: 0.86
F1: 0.7818181818181817


Class 1:

Precision: 0.825
Recall: 0.66
F1: 0.7333333333333334


Weighted avg F1: 0.7682875264270613
Macro avg F1: 0.7575757575757576


In [64]:
fn = results_perturbed[0]['fn']
fp = results_perturbed[0]['fp']
tn = results_perturbed[0]['tn']
tp = results_perturbed[0]['tp']

print('Class 0:\n')

precision = tn/(tn+fn)
print(f'Precision: {precision}')

recall = tn/(tn+fp)
print(f'Recall: {recall}')

f10 = 2*((precision*recall)/(precision+recall))
print(f'F1: {f10}')

print()
print()

print('Class 1:\n')

precision = tp/(tp+fp)
print(f'Precision: {precision}')

recall = tp/(tp+fn)
print(f'Recall: {recall}')

f11 = 2*((precision*recall)/(precision+recall))
print(f'F1: {f11}')

print()
print()

print(f'Weighted avg F1: {(620*f10+240*f11)/860}')
print(f'Macro avg F1: {0.5*f10+0.5*f11}')

Class 0:

Precision: 0.6461538461538462
Recall: 0.84
F1: 0.7304347826086957


Class 1:

Precision: 0.7714285714285715
Recall: 0.54
F1: 0.6352941176470589


Weighted avg F1: 0.7038838993635876
Macro avg F1: 0.6828644501278773
